In [ ]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import datetime
from google.colab import files
plt.style.use('fivethirtyeight')

In [ ]:
df = pd.read_csv('/content/LSTM AKBI.csv',index_col=False)
df.head()

In [ ]:
colom='Urea Prilled Bulk FOB Black Sea Spot USD/t'

In [ ]:
df=df[['Date',colom]]

In [ ]:
df.head()

In [ ]:
df['Date']=pd.to_datetime(df['Date'], infer_datetime_format= True)
indexdf=df.set_index(['Date'])

In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Average price History')
plt.plot(indexdf)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Avarage Price USD', fontsize=18)
plt.show()

In [ ]:
data = df.filter([colom])
dataset= data.values
training_data_len=math.ceil(len(dataset)*.8)
training_data_len

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
train_data = scaled_data[0:training_data_len,:]
x_train = []
y_train = [] 

for i in range (60,len(train_data)):
  x_train.append(train_data[i-60:i, 0])
  y_train.append(train_data[i,0])
  if i<= 60:
    print(x_train)
    print(y_train)
    print()

In [ ]:
x_train,y_train=np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train,(x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
#membangun model lstm
model = Sequential()
model.add(LSTM(50,return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#train the model
model.fit(x_train, y_train,batch_size=10,epochs=300)

In [ ]:
test_data= scaled_data[training_data_len-60: , :]
x_test=[]
y_test= dataset[training_data_len: , :]
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0])

In [ ]:
print(len(test_data))

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
x_test.shape

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
rmse = np.sqrt(np.mean(predictions-y_test)**2)
rmse

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions']=predictions

plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Price', fontsize=18)
plt.plot(train[colom])
plt.plot(valid[[colom,'Predictions']])
plt.legend(['Train','Val','Predictions'], loc='lower right')
plt.show()

In [ ]:
valid

In [ ]:
len(test_data)

In [ ]:
  apple_quote = pd.read_csv('/content/LSTM AKBI.csv')
  apple_quote=apple_quote[[colom]]
  apple_quote.dropna(inplace=True)

In [ ]:
new_df=apple_quote.filter([colom])
df.dropna(inplace=True)
hasil=model

In [ ]:
new_df=apple_quote.filter([colom])
df.dropna(inplace=True)
for i in range(60):
  last_60_days=new_df[-60:]  
  last_60_days_array=np.reshape(last_60_days,(-1,1))
  last_60_days_scaled= scaler.transform(last_60_days_array)
  X_test=[]
  X_test.append(last_60_days_scaled)
  X_test = np.array(X_test)
  X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
  pred_price=model.predict(X_test)
  pred_price = scaler.inverse_transform(pred_price)
  new_df=np.append(new_df,pred_price)



In [ ]:
future_pred=new_df[-60:]
future_pred

In [ ]:
new_df=apple_quote.filter([colom])
last_60_days = new_df[-60:].values
last_60_days_scaled= scaler.transform(last_60_days)
X_test=[]
X_test.append(last_60_days_scaled)
for i in range(60, len(X_test)):
  X_test.append(X_test[i-60:i, 0])
  X_test=np.array(X_test)
  X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
  Predictions = model.predict(X_test)
  X_test = X_test.append(Predictions)
  Predictions=scaler.inverse_transform(Predictions)


In [ ]:
result=pd.concat([indexdf,valid['Predictions']], axis=1)
df_date=df[['Date']]
last_date=df_date[-1:]
future_date=[]
i=7

for x in range (60):
  end_date = last_date + datetime.timedelta(days=i)
  future_date.append(end_date)
  i+=7

future_date=np.array(future_date)
future_date=future_date.reshape(-1)
future_result={'Date':future_date,'Future Prediction':future_pred}
final_df=pd.DataFrame(data=future_result)
final_result=pd.concat([df,final_df],axis=0)
final_result['Date']=pd.to_datetime(final_result['Date'], infer_datetime_format= True)
final_result=final_result[['Date','Future Prediction']]
final_result=final_result[-60:]
final_result=final_result.set_index('Date')
fix_result=pd.concat([result,final_result],axis=0)
plt.figure(figsize=(16,8))
plt.plot(fix_result)
plt.show()